In [1]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime, timedelta
import networkx as nx
import h5py

In [129]:
start = datetime(2022, 9, 11, 0, 0)
end = datetime(2022, 9, 24, 23, 59)
#end = datetime(2023, 2, 28, 23, 59)

zones = ['AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']
zones2 = ['AT', 'BE', 'CZ', 'DE', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

In [130]:
df_ptdf = pd.read_excel('../flow_based_domain/ptdf_z_obs.xlsx', index_col=0)
display(df_ptdf)

,DateTime,line_id,contingency,fref,frm,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK
0,2022-09-01 00:00:00,12,-1,177.0,42.0,153.0,-0.04836,-0.04861,-0.01764,-0.04791,-0.01287,-0.04590,-0.04244,-0.00756,0.00085,-0.05174,0.04220,0.00071,-0.01033,0.00715
1,2022-09-01 00:00:00,94,9,863.0,173.0,653.0,-0.01960,-0.02016,-0.02146,-0.01994,-0.07237,-0.03774,-0.02340,-0.00799,-0.00063,-0.01585,0.08202,0.00082,-0.01122,0.00769
2,2022-09-01 00:00:00,152,8,491.0,206.0,1140.0,-0.12361,-0.09008,0.00777,-0.13408,0.00672,-0.00222,-0.14070,-0.00395,0.00176,-0.07122,0.00364,0.00030,-0.00444,0.00304
3,2022-09-01 00:00:00,223,7,1243.0,207.0,919.0,-0.08881,-0.11644,0.01279,-0.08286,-0.02146,-0.00737,0.00638,0.00944,-0.00309,-0.09761,-0.02508,-0.00078,0.01233,-0.00938
4,2022-09-01 00:00:00,333,6,692.0,149.0,887.0,0.08342,0.08146,-0.01833,0.08480,-0.00646,0.05803,0.09712,0.00762,-0.00440,0.07500,0.00517,-0.00053,0.00870,-0.00457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299551,2023-03-31 22:00:00,2833,267,130.0,140.0,983.0,0.08168,0.08136,0.05098,0.08213,-0.07219,0.07571,0.08407,0.01285,-0.00404,0.07844,-0.00940,-0.00124,0.02356,-0.01421
299552,2023-03-31 22:00:00,2842,268,-68.0,45.0,262.0,0.04749,0.04863,-0.01230,0.04720,0.01530,0.04659,0.04495,-0.00054,-0.00184,0.04800,0.01892,0.00011,-0.00013,0.00249
299553,2023-03-31 22:00:00,2844,269,-68.0,45.0,262.0,0.04748,0.04862,-0.01230,0.04720,0.01530,0.04658,0.04495,-0.00054,-0.00184,0.04800,0.01892,0.00011,-0.00013,0.00249
299554,2023-03-31 22:00:00,2850,-1,764.0,251.0,908.0,-0.07457,-0.03530,0.00516,-0.08367,0.00179,0.00098,-0.09946,-0.00087,0.00050,-0.04448,-0.00075,0.00008,-0.00168,0.00058


In [126]:
df_ptdf_obs = df_ptdf[(df_ptdf['DateTime'] >= start) & (df_ptdf['DateTime'] <= end)]
display(df_ptdf_obs)

,DateTime,line_id,contingency,fref,frm,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK
13048,2022-09-11 00:00:00,12,-1,-47.0,42.0,300.0,-0.04779,-0.04881,-0.01929,-0.04738,-0.01285,-0.04535,-0.04328,-0.00793,0.00027,-0.05039,0.04848,0.00064,-0.01158,0.00575
13049,2022-09-11 00:00:00,383,89,214.0,177.0,1711.0,0.23039,0.46392,0.00437,0.20658,-0.01298,-0.01773,0.11721,0.00716,-0.00207,0.14913,-0.01460,-0.00054,0.01100,-0.00568
13050,2022-09-11 00:00:00,493,47,228.0,30.0,235.0,-0.05630,-0.05676,-0.05871,-0.05620,-0.02075,-0.05834,-0.05485,-0.00179,0.00315,-0.05622,-0.02102,0.00007,-0.00363,-0.00137
13051,2022-09-11 00:00:00,533,56,303.0,76.0,494.0,-0.05362,-0.05297,-0.10144,-0.05382,-0.02484,-0.05538,-0.05562,0.00724,-0.00173,-0.05258,-0.02334,-0.00059,0.01128,-0.00889
13052,2022-09-11 00:00:00,538,1,251.0,152.0,1169.0,0.14770,0.14788,0.17164,0.14768,0.15358,0.15050,0.14711,0.04929,-0.04424,0.14730,0.08921,-0.00280,0.09338,0.01564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32779,2022-09-25 23:00:00,2832,-1,589.0,155.0,1118.0,0.09389,0.09466,0.04154,0.09380,-0.08494,0.09352,0.09236,0.01222,-0.00470,0.09257,0.00207,-0.00098,0.01964,-0.01317
32780,2022-09-25 23:00:00,2833,-1,547.0,140.0,969.0,0.08080,0.08128,0.03947,0.08081,-0.07064,0.07973,0.08028,0.01152,-0.00391,0.07899,-0.00416,-0.00093,0.01823,-0.01246
32781,2022-09-25 23:00:00,2842,-1,14.0,45.0,179.0,0.04631,0.04726,-0.03560,0.04606,0.01802,0.04583,0.04360,-0.00048,-0.00186,0.04687,0.01849,0.00007,0.00078,0.00278
32782,2022-09-25 23:00:00,2844,-1,14.0,45.0,179.0,0.04631,0.04725,-0.03560,0.04605,0.01802,0.04582,0.04360,-0.00047,-0.00186,0.04687,0.01849,0.00007,0.00078,0.00278


In [81]:
df_grid = pd.read_excel('../flow_based_domain/grid.xlsx', index_col=0)
df_timestamps = pd.read_excel('../flow_based_domain/timestamps.xlsx', index_col=0)
df_line_edge_map = pd.read_excel('../flow_based_domain/line_edge_map.xlsx', index_col=0)
df_grid = df_grid[df_grid.susceptance != 0]
Grid = nx.from_pandas_edgelist(df_grid, 'substation_1', 'substation_2', edge_attr=["susceptance"], create_using=nx.MultiGraph)

In [9]:
ref_flow_obs = np.zeros((len(df_ptdf['DateTime'].dt.day.unique()), 24, len(df_grid)))
ram_obs = np.zeros((len(df_ptdf['DateTime'].dt.day.unique()), 24, len(df_grid)))

for day_index, day in enumerate(df_ptdf['DateTime'].dt.day.unique()):
    for hour in df_ptdf['DateTime'].dt.hour.unique():
        df_ptdf_hourly = df_ptdf[(df_ptdf['DateTime'].dt.day == day) & (df_ptdf['DateTime'].dt.hour == hour)]
        for index, cne in df_ptdf_hourly.iterrows():
            if len(df_grid.index[df_grid.eic == cne['cneEic']]) > 0:
                edge = df_line_edge_map.loc[df_line_edge_map.line_id == df_grid.index[df_grid.eic == cne['cneEic']][0], 'edge'].iloc[0]
                ref_flow_obs[day_index][hour][edge] = cne['fref']
                ram_obs[day_index][hour][edge] = cne['ram']
    print(day)
    
with open('ref_flow_obs.npy', 'wb') as f:
    np.save(f, ref_flow_obs)
    
with open('ram_obs.npy', 'wb') as f:
    np.save(f, ram_obs)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [11]:
hour = 0
day = 1
ref_flow_obs[day][hour][ref_flow_obs[day][hour] != 0]

array([1953.,  985., 1714., 1858., 2210.,  400., -535., 2075., -901.,
        526.,   89.,  -91.,  -91., 1220.,   93., 1169.,  -98.,  -53.,
        519.,  108.,  647., -592.,  658.,  676.,  840.,  -43., -180.,
        923., 1052.,  757., -121., -538., -608.,  582., -433., -640.,
        537.,  936., -650., -618.,  -63., -538.,  769., 1098.,   80.,
        727., -175.,  784., 1184.,  239.,  966., 2279., 1361., 1137.,
       1541., 2019.])

In [57]:
np.argwhere(ref_flow_obs[hour] != 0).flatten()

array([  15,   16,   52,  148,  187,  281,  322,  326,  377,  415,  430,
        477,  539,  542,  621,  629,  632,  634,  635,  636,  654,  659,
        664,  668,  689,  693,  702,  715,  719,  755,  757,  768,  775,
        783,  785,  792,  794,  855,  870,  877,  933,  939,  942,  945,
       1015, 1035, 1055, 1057, 1060, 1088, 1089, 1205, 1419, 2253, 2254,
       2266, 2280, 2364, 2422, 2431, 2445, 2461, 2603, 2605, 2670, 2713],
      dtype=int64)

In [154]:
hf_nuts = h5py.File("./ref_flow_nuts.jld", "r")
hf = h5py.File("./ref_flow.jld", "r")
ref_flow_nuts = np.transpose(hf_nuts.get("data")[0:np.shape(hf_nuts.get("data"))[0]][0:np.shape(hf_nuts.get("data"))[1]])
ref_flow = np.transpose(hf.get("data"))

In [156]:
num_hours = 24*(abs((end - start).days) + 1)
ref_flow_mre = np.zeros((len(zones), num_hours))
ref_flow_mae = np.zeros((len(zones), num_hours))
ref_flow_num = np.zeros((len(zones), num_hours))

for t, (t_index, timestamp) in enumerate(df_timestamps[(df_timestamps.index >= start) & (df_timestamps.index <= end)].iterrows()):
    print(t)
    df_ptdf_hourly = df_ptdf_obs[df_ptdf_obs['DateTime'] == t_index]
    for index, cne in df_ptdf_hourly.iterrows():
        l = df_line_edge_map.loc[df_line_edge_map.line_id == cne['line_id'], 'edge'].iloc[0]
        z = zones2.index(df_grid.loc[cne['line_id'], 'zone'])
        if abs(cne['fref']) > 0:
            ref_flow_mre[z][t] += abs(cne['fref'] - ref_flow[t][l]) / abs(cne['fref'])
            ref_flow_mae[z][t] += abs(cne['fref'] - ref_flow[t][l])
            ref_flow_num[z][t] += 1
        
ref_flow_mae = np.divide(ref_flow_mae, ref_flow_num)
ref_flow_mre = 100*np.divide(ref_flow_mre, ref_flow_num)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [158]:
np.mean(np.nan_to_num(ref_flow_mre))

406.8615326566512

In [159]:
ref_flow_mre

array([[ 233.92210756,  233.42448073,  225.23584882, ...,  100.89985971,
        1136.06570402,  205.73635104],
       [  84.04383977,   76.01805175,   83.60375248, ...,   71.532862  ,
          77.01433025,   41.81519861],
       [ 182.59968473,  193.94247723,  122.88347285, ...,  212.84817241,
        2221.82882153,  310.66332952],
       ...,
       [ 125.67143574,  126.59143519,  127.1228903 , ...,  109.27734375,
         143.65472561,  102.8973095 ],
       [          nan,           nan,           nan, ...,  210.93966947,
         111.90473819,   97.53518518],
       [ 114.10878155,  100.91745934,  119.58332271, ...,  114.19401118,
         115.53940487,  108.10707917]])

In [160]:
l = df_line_edge_map.loc[df_line_edge_map.line_id == 2829, 'edge'].iloc[0]
df_ptdf_line = df_ptdf_obs[df_ptdf_obs['line_id'] == 2829]
display(df_ptdf_line['fref'].mean())

np.mean(ref_flow[:][l])

627.2660377358491

IndexError: index 2461 is out of bounds for axis 0 with size 336

In [ ]:
display(df_timestamps[(df_timestamps.index >= start) & (df_timestamps.index <= end)])
print(ref_flow_nuts)

In [153]:
ref_flow = np.transpose(hf.get("data"))

print(np.shape(ref_flow))
print(24*(abs((end - start).days) + 1))
print(np.shape(hf.get("data"))[0])

(336, 2814)
336
2814
